In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

df = pd.read_csv('historic_home.csv', parse_dates=['date'])
zip_codes = df['zip'].unique()
df['date'] = pd.to_datetime(df['date'])

Data is splitted into multiple individual DataFrames to conduct specific time series analysis to determine a more accurate home price trend. 

In [ ]:
df_bb = df[df['zip'] == 24060]

df_cb = df[df['zip'] == 24073]

df_rnk_1 = df[df['zip'] == 24012]

df_rnk_2 = df[df['zip'] == 24014] 

df_rnk_3 = df[df['zip'] == 24016]

df_rnk_4 = df[df['zip'] == 24018]